In [4]:
! pip install google-cloud-aiplatform google-cloud-discoveryengine langchain==0.0.251 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 nemoguardrails==0.5.0 --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 918.2/918.2 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 56.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

In [11]:
import os

# NeMo canonical form and flow
canonical_text="""
define user express greeting
  "Hello"
  "Hi"
  "Wassup?"

define bot express greeting
  "Hey there!"

define bot ask how are you
  "How are you doing?"
  "How's it going?"
  "How are you feeling today?"

define flow greeting
  user express greeting
  bot express greeting
  bot ask how are you

  when user express feeling good
   bot express positive emotion

  else when user express feeling bad
   bot express empathy

define user ask about politics
  "What do you think about the government?"
  "Which party should I vote for?"

define user ask about stock market
  "Which stock should I invest in?"
  "Would this stock 10x over the next year?"

define flow politics
  user ask about politics
  bot inform cannot respond

define flow stock market
  user ask about stock market
  bot inform cannot respond
"""

# Specify the directory name
directory_name = "./config"

# Specify the file name
file_name = "hello.co"

# Combine the directory and file paths
path = os.path.join(directory_name, file_name)

# Create the directory if it doesn't exist
if not os.path.exists(directory_name):
    os.makedirs(directory_name)

# Write text content to the file
with open(path, 'w') as file:
    file.write(canonical_text)

print(f"Directory '{directory_name}' and file '{file_name}' created successfully.")

Directory './config' and file 'hello.co' created successfully.


In [12]:
import os
import vertexai
from langchain.llms import VertexAI
from nemoguardrails import LLMRails, RailsConfig

vertexai.init(project="<project_id>", location="us-central1")

# Load configuration
config = RailsConfig.from_path("./config")

llm = VertexAI(
    model_name="text-bison",
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
)

# Configuration of LLMs is passed
app = LLMRails(config=config, llm=llm)

In [13]:
# test user input
new_message = app.generate(messages=[{
    "role": "user",
    "content": "Hello!"
}])
print(f"new_message: {new_message}")

# test user input
new_message = app.generate(messages=[{
    "role": "user",
    "content": "Would this stock 10x over the next year?"
}])
print(f"new_message: {new_message}")

# test user input
new_message = app.generate(messages=[{
    "role": "user",
    "content": "Which party should I vote for?"
}])
print(f"new_message: {new_message}")

new_message: {'role': 'assistant', 'content': 'Hey there!\nHow are you feeling today?'}
new_message: {'role': 'assistant', 'content': 'I cannot provide financial advice, but I can help you find information about stocks and the stock market.'}
new_message: {'role': 'assistant', 'content': 'I am not able to provide political opinions or recommendations. However, I can provide you with information about the different political parties and their platforms so that you can make an informed decision.'}
